In [1]:
import os
from llama_parse import LlamaParse
from llama_parse.base import ResultType
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from llama_cloud_services.parse.utils import Language
from langchain_community.embeddings.ollama import OllamaEmbeddings

In [2]:


os.environ["LLAMA_CLOUD_API_KEY"] = "llx-a2C7FgYfP1hzX3pXuvtdaNmexAqsuRnJIJ2G6MjbBrfuS3QY"  # Remplace par ta clé API

parser_fr = LlamaParse(result_type=ResultType.MD, language=Language.FRENCH)
parser_en = LlamaParse(result_type=ResultType.MD, language=Language.ENGLISH)


In [3]:

import nest_asyncio
nest_asyncio.apply()

pdf_files = [
    ("philosophie.pdf", parser_fr)
   
    
    # ajoute autant que nécessaire
]

# Nom du fichier Markdown de sortie
output_filename = "plagia_data.md"

# Traitement et sauvegarde
with open(output_filename, 'w', encoding='utf-8') as f:
    for file_name, parser in pdf_files:
        print(f"Traitement de {file_name}...")
        documents = parser.load_data(file_name)
        f.write(f"# Contenu extrait de : {file_name}\n\n")
        for doc in documents:
            f.write(doc.text + "\n\n")

print(f"✅ Tous les textes ont été extraits et sauvegardés dans : {output_filename}")


Traitement de philosophie.pdf...
Started parsing the file under job_id bcb1ab79-e166-4be6-918c-19ee9e70e745
.✅ Tous les textes ont été extraits et sauvegardés dans : plagia_data.md


In [4]:
# 1. Lecture du fichier Markdown
with open("plagia_data.md", encoding='utf-8') as f:
    markdown_content = f.read()

# 2. Découpage par paragraphes (2 sauts de ligne ou plus)
paragraphs = [p.strip() for p in markdown_content.split('\n\n') if p.strip()]

# 3. Création des documents
documents = [Document(page_content=paragraph) for paragraph in paragraphs]

# 4. Initialisation des embeddings
embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")

# 5. Configuration de la base vectorielle
persist_directory = "philo_db"
vecdb = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_directory,
    collection_name="rag-chroma"
)

# 6. Persistance des données
vecdb.persist()
print("Opération terminée avec succès:")
print(f"- {len(documents)} paragraphes traités")
print(f"- Base vectorielle sauvegardée dans: {persist_directory}")

C:\Users\hiba\AppData\Local\Temp\ipykernel_10004\1223247682.py:12: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="mxbai-embed-large:latest")


Opération terminée avec succès:
- 914 paragraphes traités
- Base vectorielle sauvegardée dans: philo_db


C:\Users\hiba\AppData\Local\Temp\ipykernel_10004\1223247682.py:24: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vecdb.persist()
